In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
! pip install datasets transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.9 MB/s eta 0:00:00


In [ ]:
! pip install datasets

In [ ]:
! pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from datasets import Dataset, Features, Value, ClassLabel

In [ ]:
def load_data(file_path):
    df = pd.read_csv(file_path)
    df.rename(columns=lambda x: x.strip(), inplace=True)
    df = df[['text', 'labels']]
    return df

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import AutoModelForSequenceClassification

initial_final = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/nlp_project_MODELS/final_model')
initial_checkpoint = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/nlp_project_MODELS/checkpoint-11725')

fully_balanced_checkpoint = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP_homo_trans_project_models/fully_balanced/checkpoint-22596')
fully_balanced = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP_homo_trans_project_models/fully_balanced/checkpoint-37660')

backtranslate_checkpoint = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP_homo_trans_project_models/without_OS/checkpoint-18760')
backtranslate_final = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/NLP_homo_trans_project_models/without_OS/final_model')

all_models = [initial_final, initial_checkpoint, backtranslate_final, backtranslate_checkpoint, fully_balanced, fully_balanced_checkpoint]

In [ ]:
from transformers import Trainer
from sklearn.metrics import accuracy_score
import numpy as np
from datasets import load_metric
from datasets import load_dataset
from evaluate import evaluator
from transformers import AutoModelForSequenceClassification, pipeline

In [ ]:
def load_test_data(file_path):
    df = pd.read_csv(file_path)
    df.rename(columns=lambda x: x.strip(), inplace=True)
    df.rename(columns={'labels': 'label'}, inplace=True)
    df = df[['text', 'label']]
    return df

In [ ]:
test_eng_with_labels = load_test_data('/content/drive/MyDrive/NLP_homo_trans_project_models/test_datasets/Test_Tulu_with_label.csv')

In [ ]:
# def compute_metrics(eval_pred):
#     metric1 = evaluate.load("precision")
#     metric2 = evaluate.load("recall")
#     metric3 = evaluate.load("f1")
#     metric4 = evaluate.load("accuracy")

#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)

#     precision = metric1.compute(predictions=predictions, references=labels,
#                                 average="macro")["precision"]
#     recall = metric2.compute(predictions=predictions, references=labels,
#                              average="macro")["recall"]
#     f1 = metric3.compute(predictions=predictions, references=labels,
#                          average="macro")["f1"]
#     accuracy = metric4.compute(predictions=predictions, references=labels)[
#         "accuracy"]

#     return {"precision": precision, "recall": recall, "f1": f1,
#             "accuracy": accuracy}

In [ ]:
class_names = ['Homophobia', 'Transphobia']

In [ ]:
# test_set = Dataset.from_pandas(test_eng_with_labels, features=Features({'text': Value('string')}))
test_eng_with_labels = Dataset.from_pandas(test_eng_with_labels, features=Features({'text': Value('string'), 'label': ClassLabel(num_classes=3, names=class_names)}))

In [ ]:
print(test_eng_with_labels)

Dataset({
    features: ['text', 'label'],
    num_rows: 379
})


In [ ]:
from transformers import AutoTokenizer, DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
checkpoint = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset_test = test_eng_with_labels.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

In [ ]:
data = test_eng_with_labels
task_evaluator = evaluator("text-classification")

In [ ]:
class ConfiguredMetric:
    def __init__(self, metric, *metric_args, **metric_kwargs):
        self.metric = metric
        self.metric_args = metric_args
        self.metric_kwargs = metric_kwargs

    def add(self, *args, **kwargs):
        return self.metric.add(*args, **kwargs)

    def add_batch(self, *args, **kwargs):
        return self.metric.add_batch(*args, **kwargs)

    def compute(self, *args, **kwargs):
        return self.metric.compute(*args, *self.metric_args, **kwargs, **self.metric_kwargs)

    @property
    def name(self):
        return self.metric.name

    def _feature_names(self):
        return self.metric._feature_names()

In [ ]:
from datasets import load_metric
import evaluate

for model in all_models:
    eval_results = task_evaluator.compute(
      model_or_pipeline=model,
      data=tokenized_dataset_test,
      metric=evaluate.combine([
              evaluate.load('accuracy'),
              ConfiguredMetric(evaluate.load('f1'), average='macro'),
              ConfiguredMetric(evaluate.load('precision'), average='macro'),
              ConfiguredMetric(evaluate.load('recall'), average='macro'),
          ]),
      tokenizer=tokenizer,
      label_mapping={"LABEL_0": 0, "LABEL_1": 1, "LABEL_2": 2}

  )
    print(eval_results)

{'accuracy': 0.820580474934037, 'f1': 0.4648671096345515, 'precision': 0.5789007092198581, 'recall': 0.504257558362036, 'total_time_in_seconds': 16.116234951000024, 'samples_per_second': 23.516658894109927, 'latency_in_seconds': 0.04252304736411616}
{'accuracy': 0.8153034300791556, 'f1': 0.3372706039372706, 'precision': 0.47054965236783425, 'recall': 0.3457551983671387, 'total_time_in_seconds': 16.27676631999975, 'samples_per_second': 23.284723301231605, 'latency_in_seconds': 0.042946612981529686}
{'accuracy': 0.8364116094986808, 'f1': 0.35083509571064475, 'precision': 0.6136567834681043, 'recall': 0.3582089552238806, 'total_time_in_seconds': 16.409426801000336, 'samples_per_second': 23.09648012670959, 'latency_in_seconds': 0.043296640635884794}
{'accuracy': 0.8311345646437994, 'f1': 0.3313370763152423, 'precision': 0.6106666666666666, 'recall': 0.34825870646766166, 'total_time_in_seconds': 16.18022741599998, 'samples_per_second': 23.423650994251297, 'latency_in_seconds': 0.04269189291